In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentimenttraindataset/train_2kmZucJ.csv


In [15]:
df = pd.read_csv('/kaggle/input/sentimenttraindataset/train_2kmZucJ.csv')
df.sample(5)

,id,label,tweet
2834,2835,0,Love kisses customized case for the iphone 3 f...
6838,6839,0,99% #lowfat #yogurt #apple and peach #yogurt #...
1155,1156,0,Gain Followers RT This MUST FOLLOW ME I FOLLOW...
1655,1656,0,When dreams come true. #macbook #macbookpro #a...
1134,1135,0,Listening to #win #god #iphone #android #ps3 #...


In [16]:
from IPython.core.display import HTML
HTML(df.sample(10).to_html(index=False))

id,label,tweet
738,0,Me n my bridesmaid #white #wedding #iphone #instagram #bride #bridesmaid #memories #memory #forever o http://instagr.am/p/RAxoBXPKWI/
7750,0,My second DIY phone case Just a tad proud ;) #iphone #iphonecase #case #diy #blue #purple #me http://instagram.com/p/bCLdaMxBE_/
3878,0,#post #apple #dtm #share #sexy #lingerie #woman #porno #breast #speedway #boobs #$&@*# #real #new Sexy Girls==&gt; http://bit.ly/LUQ5nw
4236,1,Get my back fixe on my iphone and drop it a cracked it already #cheep rself #apple #$&@*# #merked
3120,1,.@MacWorld Loving my #iPad EXCEPT for fact it won't play videos from 1/2 the sites I visit as I can't use #Adobe Flash! MAJOR . #Apple!
5636,0,Ahhh i'm so lucky! :3 #laptop #mac #macbookpro #apple #pretty #new #excited http://instagr.am/p/PKC66zjdWD/
4174,0,Got this from #samsung hahaha it was free.. me likeyy♡ #free #handphone #samsung #purple #android #in http://instagr.am/p/OJadRKkoke/
3358,0,@TeambringitMike iTunes stores are quite the guys at that aren't they.
4037,0,Road to platino #Danganronpa #Trigger #Havoc #Platino #PsVita #Sony #Trofei http://instagram.com/p/xUDBQvlycn/
4565,0,Just for fun #instagram #instafun #funny #myself #me #android #samsung #androidnesia http://instagram.com/p/XR_7BXijsr/


In [17]:
df = df.drop(columns=['id'])
df.head(15)

,label,tweet
0,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,0,Finally a transparant silicon case ^^ Thanks t...
2,0,We love this! Would you go? #talk #makememorie...
3,0,I'm wired I know I'm George I was made that wa...
4,1,What amazing service! Apple won't even talk to...
5,1,iPhone software update fucked up my phone big ...
6,0,Happy for us .. #instapic #instadaily #us #son...
7,0,New Type C charger cable #UK http://www.ebay.c...
8,0,Bout to go shopping again listening to music #...
9,0,Photo: #fun #selfie #pool #water #sony #camera...


In [18]:
# lowercasing
df['tweet'] = df['tweet'].str.lower()

In [19]:
# remove URL
import re
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [20]:
df['tweet']=df['tweet'].apply(remove_url)

In [21]:
# Remove Hashtags
import re
def remove_hashtags(text):
    return re.sub(r'#\w*', '', text)
df['tweet'] = df['tweet'].apply(remove_hashtags)
print(df['tweet'])

0                                          test          
1       finally a transparant silicon case ^^ thanks t...
2                 we love this! would you go?        ... 
3       i'm wired i know i'm george i was made that wa...
4       what amazing service! apple won't even talk to...
                              ...                        
7915                                live out loud        
7916    we would like to wish you an amazing day! make...
7917    helping my lovely 90 year old neighbor with he...
7918    finally got my    stay connected anytime,anywh...
7919                       apple barcelona!!!          … 
Name: tweet, Length: 7920, dtype: object


In [22]:
contractions_dict = {
    "i'm": "I am",
    "you're": "you are",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "we're": "we are",
    "they're": "they are",
    "don't": "do not",
    "doesn't": "does not",
    "can't": "cannot",
    "couldn't": "could not",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "won't": "will not",
    "would've": "would have",
    "might've": "might have",
    "must've": "must have",
    "shan't": "shall not",
    "let's": "let us",
    "o'clock": "of the clock",
    "y'all": "you all",
}

In [23]:
import re
def expand_contractions(text):
    # Replace contractions in the text
    pattern = re.compile(r'\b(' + '|'.join(contractions_dict.keys()) + r')\b', re.IGNORECASE)
    expanded_text = pattern.sub(lambda x: contractions_dict[x.group(0).lower()], text)
    
    return expanded_text

# Exam/

In [24]:
df['tweet'] = df['tweet'].apply(expand_contractions)
df['tweet']

0                                          test          
1       finally a transparant silicon case ^^ thanks t...
2                 we love this! would you go?        ... 
3       I am wired i know I am george i was made that ...
4       what amazing service! apple will not even talk...
                              ...                        
7915                                live out loud        
7916    we would like to wish you an amazing day! make...
7917    helping my lovely 90 year old neighbor with he...
7918    finally got my    stay connected anytime,anywh...
7919                       apple barcelona!!!          … 
Name: tweet, Length: 7920, dtype: object

In [25]:
# Remove punctuation
import string 
exclude = string.punctuation
exclude
def removePunc(text):
    return text.translate(str.maketrans('','',exclude))
df['tweet']=df['tweet'].apply(removePunc)

In [26]:
HTML(df.head(5).to_html(index=False))

label,tweet
0,test
0,finally a transparant silicon case thanks to my uncle …
0,we love this would you go
0,I am wired i know I am george i was made that way
1,what amazing service apple will not even talk to me about a question i have unless i pay them 1995 for their stupid support


In [27]:
# Tokenizer
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
df['tweet_tokens'] = df['tweet'].apply(tknzr.tokenize)
print(df['tweet_tokens'])

0                                                  [test]
1       [finally, a, transparant, silicon, case, thank...
2                        [we, love, this, would, you, go]
3       [i, am, wired, i, know, i, am, george, i, was,...
4       [what, amazing, service, apple, will, not, eve...
                              ...                        
7915                                    [live, out, loud]
7916    [we, would, like, to, wish, you, an, amazing, ...
7917    [helping, my, lovely, 90, year, old, neighbor,...
7918    [finally, got, my, stay, connected, anytimeany...
7919                                [apple, barcelona, …]
Name: tweet_tokens, Length: 7920, dtype: object


In [28]:
# Remove Stop Words
from nltk.corpus import stopwords
cache_english_stopwords = stopwords.words('english')
df['tweet'] = df['tweet_tokens'].apply(lambda x: [i for i in x if i not in cache_english_stopwords])
print( df['tweet'])

0                                                  [test]
1       [finally, transparant, silicon, case, thanks, ...
2                                       [love, would, go]
3                        [wired, know, george, made, way]
4       [amazing, service, apple, even, talk, question...
                              ...                        
7915                                         [live, loud]
7916    [would, like, wish, amazing, day, make, every,...
7917    [helping, lovely, 90, year, old, neighbor, ipa...
7918     [finally, got, stay, connected, anytimeanywhere]
7919                                [apple, barcelona, …]
Name: tweet, Length: 7920, dtype: object


In [29]:
# Filter the Data after Preprocess
df['tweet_filtered'] = df['tweet'].apply(lambda x: ' '.join(x))
print( df['tweet_filtered'])

0                                                    test
1         finally transparant silicon case thanks uncle …
2                                           love would go
3                              wired know george made way
4       amazing service apple even talk question unles...
                              ...                        
7915                                            live loud
7916    would like wish amazing day make every minute ...
7917    helping lovely 90 year old neighbor ipad morni...
7918           finally got stay connected anytimeanywhere
7919                                    apple barcelona …
Name: tweet_filtered, Length: 7920, dtype: object


In [31]:
from collections import Counter
import re

# Vocabulary and probabilities
vocabulary = {'example', 'condition', 'during', 'generation', 'modified', 'same', 'manner'}
word_probabilities = {'example': 0.01, 'condition': 0.02, 'during': 0.03, 'generation': 0.01, 
                      'modified': 0.01, 'same': 0.02, 'manner': 0.01}

# Edit distance functions
def edit1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edit2(word): 
    return (e2 for e1 in edit1(word) for e2 in edit1(e1))

# Spelling correction function
def correct_spelling(word):
    if word in vocabulary:
        return word
    suggestions = edit1(word) or edit2(word) or [word]
    best_guesses = [w for w in suggestions if w in vocabulary]
    return max(best_guesses, key=word_probabilities.get, default=word)

# Apply correction
df['tweet_corrected'] = df['tweet'].apply(
    lambda tokens: [correct_spelling(word) for word in tokens]
)

print(df['tweet_corrected'])


0                                                  [test]
1       [finally, transparant, silicon, case, thanks, ...
2                                       [love, would, go]
3                        [wired, know, george, made, way]
4       [amazing, service, apple, even, talk, question...
                              ...                        
7915                                         [live, loud]
7916    [would, like, wish, amazing, day, make, every,...
7917    [helping, lovely, 90, year, old, neighbor, ipa...
7918     [finally, got, stay, connected, anytimeanywhere]
7919                                [apple, barcelona, …]
Name: tweet_corrected, Length: 7920, dtype: object


In [33]:
print(df['tweet'])

0                                                  [test]
1       [finally, transparant, silicon, case, thanks, ...
2                                       [love, would, go]
3                        [wired, know, george, made, way]
4       [amazing, service, apple, even, talk, question...
                              ...                        
7915                                         [live, loud]
7916    [would, like, wish, amazing, day, make, every,...
7917    [helping, lovely, 90, year, old, neighbor, ipa...
7918     [finally, got, stay, connected, anytimeanywhere]
7919                                [apple, barcelona, …]
Name: tweet, Length: 7920, dtype: object


In [35]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

def lemmatize_words_spacy(words):
    doc = nlp(" ".join(words))
    return [token.lemma_ for token in doc]

df['tweet_lemmatized'] = df['tweet'].apply(lemmatize_words_spacy)

In [36]:
# Display the DataFrame with lemmatized tweets
print(df['tweet'])
print(df['tweet_lemmatized'])

0                                                  [test]
1       [finally, transparant, silicon, case, thanks, ...
2                                       [love, would, go]
3                        [wired, know, george, made, way]
4       [amazing, service, apple, even, talk, question...
                              ...                        
7915                                         [live, loud]
7916    [would, like, wish, amazing, day, make, every,...
7917    [helping, lovely, 90, year, old, neighbor, ipa...
7918     [finally, got, stay, connected, anytimeanywhere]
7919                                [apple, barcelona, …]
Name: tweet, Length: 7920, dtype: object
0                                                  [test]
1       [finally, transparant, silicon, case, thank, u...
2                                       [love, would, go]
3                         [wire, know, george, make, way]
4       [amazing, service, apple, even, talk, question...
                              .

In [37]:
nlp = spacy.load("en_core_web_sm")

def lemmatize_chatwords(words):
    doc = nlp(" ".join(words))
    return [token.lemma_ for token in doc]

df['chatwords_lemmatized'] = df['tweet_lemmatized'].apply(lemmatize_chatwords)

In [38]:
# Display the DataFrame with lemmatized chat words
print(df['tweet'])
print(df['chatwords_lemmatized'])
df['tweet'] = df['chatwords_lemmatized'];
df

0                                                  [test]
1       [finally, transparant, silicon, case, thanks, ...
2                                       [love, would, go]
3                        [wired, know, george, made, way]
4       [amazing, service, apple, even, talk, question...
                              ...                        
7915                                         [live, loud]
7916    [would, like, wish, amazing, day, make, every,...
7917    [helping, lovely, 90, year, old, neighbor, ipa...
7918     [finally, got, stay, connected, anytimeanywhere]
7919                                [apple, barcelona, …]
Name: tweet, Length: 7920, dtype: object
0                                                  [test]
1       [finally, transparant, silicon, case, thank, u...
2                                       [love, would, go]
3                         [wire, know, george, make, way]
4       [amazing, service, apple, even, talk, question...
                              .

,label,tweet,tweet_tokens,tweet_filtered,tweet_corrected,tweet_lemmatized,chatwords_lemmatized
0,0,[test],[test],test,[test],[test],[test]
1,0,"[finally, transparant, silicon, case, thank, u...","[finally, a, transparant, silicon, case, thank...",finally transparant silicon case thanks uncle …,"[finally, transparant, silicon, case, thanks, ...","[finally, transparant, silicon, case, thank, u...","[finally, transparant, silicon, case, thank, u..."
2,0,"[love, would, go]","[we, love, this, would, you, go]",love would go,"[love, would, go]","[love, would, go]","[love, would, go]"
3,0,"[wire, know, george, make, way]","[i, am, wired, i, know, i, am, george, i, was,...",wired know george made way,"[wired, know, george, made, way]","[wire, know, george, make, way]","[wire, know, george, make, way]"
4,1,"[amazing, service, apple, even, talk, question...","[what, amazing, service, apple, will, not, eve...",amazing service apple even talk question unles...,"[amazing, service, apple, even, talk, question...","[amazing, service, apple, even, talk, question...","[amazing, service, apple, even, talk, question..."
...,...,...,...,...,...,...,...
7915,0,"[live, loud]","[live, out, loud]",live loud,"[live, loud]","[live, loud]","[live, loud]"
7916,0,"[would, like, wish, amazing, day, make, every,...","[we, would, like, to, wish, you, an, amazing, ...",would like wish amazing day make every minute ...,"[would, like, wish, amazing, day, make, every,...","[would, like, wish, amazing, day, make, every,...","[would, like, wish, amazing, day, make, every,..."
7917,0,"[help, lovely, 90, year, old, neighbor, ipad, ...","[helping, my, lovely, 90, year, old, neighbor,...",helping lovely 90 year old neighbor ipad morni...,"[helping, lovely, 90, year, old, neighbor, ipa...","[help, lovely, 90, year, old, neighbor, ipad, ...","[help, lovely, 90, year, old, neighbor, ipad, ..."
7918,0,"[finally, get, stay, connected, anytimeanywhere]","[finally, got, my, stay, connected, anytimeany...",finally got stay connected anytimeanywhere,"[finally, got, stay, connected, anytimeanywhere]","[finally, get, stay, connected, anytimeanywhere]","[finally, get, stay, connected, anytimeanywhere]"


In [39]:
# Dropping the unnecessary columns
df.drop(columns=['tweet_tokens', 'tweet_filtered','tweet_corrected','tweet_lemmatized','chatwords_lemmatized'], inplace=True)


In [40]:
df

,label,tweet
0,0,[test]
1,0,"[finally, transparant, silicon, case, thank, u..."
2,0,"[love, would, go]"
3,0,"[wire, know, george, make, way]"
4,1,"[amazing, service, apple, even, talk, question..."
...,...,...
7915,0,"[live, loud]"
7916,0,"[would, like, wish, amazing, day, make, every,..."
7917,0,"[help, lovely, 90, year, old, neighbor, ipad, ..."
7918,0,"[finally, get, stay, connected, anytimeanywhere]"


In [41]:
# from sklearn.model_selection import train_test_split

# # Splitting the data into training, validation, and test sets
# X = df['tweet']  # Features
# y = df['label']  # Labels

# # Split into 80% train + validation and 20% test
# X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Split the 80% into 70% training and 10% validation
# X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=42)

# X_train, X_valid, X_test, y_train, y_valid, y_test

(7170    [care, cover, spend, enough, money, buying, th...
 7542    [gain, follower, rt, must, follow, follow, bac...
 1008                        [thank, followcheck, website]
 2861                      [look, get, little, brother, …]
 4148    [remove, group, text, everyone, update, iphone...
                               ...                        
 6537    [instapic, mimmoal, feb, 2016, take, …, pictwi...
 1603    [lemme, get, straight, charger, suck, therefor...
 4569                           [welcome, life, new, baby]
 4784    [well, mypad, facebook, app, thing, tweet, com...
 4047                   [worry, pictwittercom, 0wqftykprt]
 Name: tweet, Length: 5544, dtype: object,
 6017    [like, share, want, diamond, case, iphone, 6, ...
 7301                     [look, death, ya, know, serious]
 6355                                   [happy, friday, …]
 2155          [frustration, deal, novice, googler, video]
 5298    [good, christmas, gift, could, ever, ask, love...
             

In [44]:
import pandas as pd
from gensim.models import Word2Vec

sentences = df['tweet'].tolist()
# df = pd.DataFrame(sentences);

In [45]:
from gensim.models import Word2Vec

# Example of training Word2Vec
word2vec_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)


(454086, 523680)

In [46]:
word2vec_model.wv.similarity(w1='good',w2="bad")

0.9852862

In [47]:
import numpy as np

#X_train, X_valid, X_test

# Function to convert a sentence to a sequence of vectors
def sentence_to_vector(sentence, model, max_len):
    sentence_vector = [model.wv[word] if word in model.wv else np.zeros(model.vector_size) for word in sentence]
    return sentence_vector[:max_len] + [np.zeros(model.vector_size)] * (max_len - len(sentence_vector))

# Set the maximum length for padding/truncating sentences
max_len = max(len(sentence) for sentence in sentences)

# Convert the sentences in each dataset to vectors
X_train_vec = np.array([sentence_to_vector(sentence, word2vec_model, max_len) for sentence in X_train])
X_valid_vec = np.array([sentence_to_vector(sentence, word2vec_model, max_len) for sentence in X_valid])
X_test_vec = np.array([sentence_to_vector(sentence, word2vec_model, max_len) for sentence in X_test])

# Check the shape of the vectors
print("Shape of X_train_vec:",  X_train_vec.shape)
print("Shape of X_valid_vec:", X_valid_vec.shape)
print("Shape of X_test_vec:", X_test_vec.shape)

Shape of X_train_vec: (5544, 33, 100)
Shape of X_valid_vec: (792, 33, 100)
Shape of X_test_vec: (1584, 33, 100)


In [48]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense,Embedding

# model = Sequential()
# model.add(LSTM(100, input_shape=(X_train_vec.shape[1], X_train_vec.shape[2])))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(X_train_vec, y_train, epochs=10, batch_size=64, validation_split=0.2)

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
import numpy as np

# Define the model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train_vec.shape[1], X_train_vec.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train_vec, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Extract validation data used during training
val_size = int(0.2 * len(X_train_vec))  # 20% validation split
X_val = X_train_vec[-val_size:]
y_val = y_train[-val_size:]

# Predict probabilities and threshold them at 0.5
y_pred_probs = model.predict(X_val)
y_pred = np.round(y_pred_probs).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_val, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_val, y_pred, average='binary')

# Print metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1_score:.4f}')

# Alternatively, detailed classification report
report = classification_report(y_val, y_pred)
print(report)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7972 - loss: 0.5335 - val_accuracy: 0.8431 - val_loss: 0.3449
Epoch 2/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8478 - loss: 0.3434 - val_accuracy: 0.8494 - val_loss: 0.3329
Epoch 3/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8437 - loss: 0.3482 - val_accuracy: 0.8566 - val_loss: 0.3258
Epoch 4/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8394 - loss: 0.3579 - val_accuracy: 0.8575 - val_loss: 0.3231
Epoch 5/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8357 - loss: 0.3518 - val_accuracy: 0.8539 - val_loss: 0.3325
Epoch 6/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8519 - loss: 0.3433 - val_accuracy: 0.8485 - val_loss: 0.3322
Epoch 7/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8441 - loss: 0.3394 - val_accuracy: 0.8611 - val_loss: 0.3212
Epoch 8/10
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8491 - loss: 0.3271 - val_accuracy: 0.8611 - val_loss

In [55]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test_vec, y_test)
print(f'Test Accuracy: {test_acc}')

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8845 - loss: 0.2949
Test Accuracy: 0.8642676472663879


In [56]:
# Evaluate the model on the test set
test_loss, train_acc = model.evaluate(X_train_vec,y_train)
print(f'Test Accuracy: {train_acc}')

174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8542 - loss: 0.3178
Test Accuracy: 0.8551587462425232


In [ ]:
# x = np.array(df['tweet'].values)
# y = np.array(df["label"].values)

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)


In [ ]:
# x_test.shape

In [ ]:
# model = Sequential()
# vocalbulary_size = 6336
# embeded_vector_size = 35
# max_length=100
# model.add(Embedding(vocalbulary_size,embeded_vector_size,input_length=max_length))
# model.add(LSTM(100))
# model.add(Dense(1,activation="sigmoid"))

# model.compile(optimizer='adam', loss='binary_crossentropy',metrics=["accuracy"])

# print(model.summary())
# print("Model Creation Completed !")